<a href="https://colab.research.google.com/github/kassabronald/Competitive-Programming/blob/main/off.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import csv
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,SGD

from google.colab import drive
drive.mount('/content/drive')


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #if you have a GPU with CUDA installed, this may speed up computation


# Load the training data
data = pd.read_csv("/content/drive/MyDrive/DATA_ML/train.csv")





#test_data = data[50000:100000]

#train_data = data[0:50000]

test_data = data[int(2*len(data)/3):]




train_data = data[0:int(2*len(data)/3)]


def getXY(df):

    
    X_num = df.loc[:, ~df.columns.isin(['text', 'urls','hashtags','mentions','timestamp','retweets_count','TweetID'])]
    X_text = df.loc[:, ['text']]
    y = df.loc[:,['retweets_count']]
    
    return X_num,X_text,y


X_train_n, X_train_t, y_train = getXY(train_data)

X_test_n, X_test_t, y_test = getXY(test_data)

meansTr =  [X_train_n[col].mean() for col in X_train_n]
meansTe  = [X_test_n[col].mean() for col in X_test_n]
StdTr = [X_train_n[col].std() for col in X_train_n]
StdTe  = [X_test_n[col].std() for col in X_test_n]


X_train_n = (X_train_n - meansTr)/StdTr

X_test_n = (X_test_n - meansTe)/StdTe



def FeedForwardNN(in_dim,out_dim):
    model = Sequential()
    #model.add(Dense(input_layer_neurons=16, hidden_layer_neurons=32, kernel_initializer='normal', activation='relu'))
    model.add(Dense(64,input_dim=in_dim,activation='tanh',kernel_initializer='he_normal'))
    model.add(Dense(64,activation='tanh'))
    model.add(Dense(64,activation='tanh'))
    model.add(Dense(64,activation='tanh'))
    # model.add(Dense(out_dim,input_dim=in_dim,activation='linear'))
    model.add(Dense(out_dim,activation='linear'))
    
    return model
    
model = FeedForwardNN(X_train_n.shape[1],y_train.shape[1])
opt = Adam(learning_rate=5*1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

model.fit(x=X_train_n, y=y_train, validation_data=(X_test_n, y_test),epochs=30, batch_size=512)

prediction =model.predict(X_test_n)
#prediction = [int(value) if value >= 0 else 0 for value in prediction]
# print([i for i in prediction if i > 0])
# print(y_test.head(10))
print("Prediction error:", mean_absolute_error(y_true=y_test, y_pred=prediction))



# # Dump the results into a file that follows the required Kaggle template
# with open("Official_pred.txt", 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(["TweetID", "retweets_count"])
#     for index, prediction in enumerate(prediction):
#         writer.writerow([str(eval_data['TweetID'].iloc[index]) , str(int(prediction))])




# diff = (prediction.flatten()) - (np.array(y_test))
# epsilon=0.001
# absolutePercentDiff = np.abs((diff/ (0.001+np.array(y_test)))*100)

# mean = np.mean(absolutePercentDiff)
# std = np.std(absolutePercentDiff)


# print(mean, std)
    
    
# modelN = FeedForwardNN(X_train_n.shape[1],5,y_train.shape[1])


# loss_function = nn.MSELoss()
# optimizer = torch.optim.SGD(modelN.parameters(), lr=0.1) #actually SGD is just GD in this case

# losses = []
# tr_acc = []
# test_acc = []


# X_train_n = (torch.from_numpy(X_train_n.to_numpy()).float().to(device))
# X_test_n = (torch.from_numpy(X_test_n.to_numpy()).float().to(device))
# y_train = (torch.from_numpy(y_train.to_numpy()).float().to(device))
# y_test = (torch.from_numpy(y_test.to_numpy()).float().to(device))

# for epoch in range(100):
#     output=modelN.forward(X_train_n)
#     loss = loss_function(output, y_train)
#     optimizer.zero_grad() #required since pytorch accumulates the gradients
#     loss.backward() #backpropagation step
#     optimizer.step() #update the parameters
    
#     #update loss and accuracy
#     losses.append(loss.data)
#     output_te=modelN(X_test_n)
#     tr_acc.append(abs(y_train.detach().numpy()-output.detach().numpy()).mean())
#     test_acc.append(abs(y_test.detach().numpy()-output_te.detach().numpy()).mean())

# #     tr_acc.append(accuracy_score(y_train.detach().numpy().astype(np.float32), output.detach().numpy().astype(np.float32)))
# #     test_acc.append(accuracy_score(y_test.detach().numpy().astype(np.float32), output_te.detach().numpy().astype(np.float32)))

    

    
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,8))
# ax1.plot(losses)
# ax1.set_title("Training Loss")
# ax1.set_xlabel("Iterations")
# ax2.plot(test_acc, c='r', label='test')
# ax2.plot(tr_acc,  c='b', label='train')
# ax2.set_title("Train and test accuracy")
# ax2.set_xlabel("Iterations")
# plt.legend()
# plt.show()

# print(losses[-1])










    

# X_train_t, X_test_t, y_train_t, y_test_t = scsplit(text, text['retweets_count'], stratify=text['retweets_count'], train_size=0.7, test_size=0.3)

# X_train_t = X_train_t.drop(['retweets_count'], axis=1)
# X_test_t = X_test_t.drop(['retweets_count'], axis=1)
# X_train_n = X_train_n.drop(['retweets_count'], axis=1)
# X_test_n = X_test_n.drop(['retweets_count'], axis=1)


# def createMLP(dimension):
    
#     model = Sequential()
    
#     model.add(Dense(8, input_dim=dimension, activation="relu"))
#     model.add(Dense(4, activation="relu"))
    
#     return model



# def preProcessText(X_text_train, X_text_test):
    
#     max_length = 1000
#     tokenizer = Tokenizer(num_words = 1000)
    
    
    
    
    
#     tokenizer.fit_on_texts(X_text_train['text'])
    
#     embeddings_index = {}
    
    
#     word_index = tokenizer.word_index
    
#     X_text_train_sequences = tokenizer.texts_to_sequences(X_text_train['text'])
    
   

#     # Use pad_sequences to transforms a list (of length num_samples) of sequences (lists of integers) into a 2D Numpy array of shape (num_samples, num_timesteps)
#     X_text_train_padded = pad_sequences(X_text_train_sequences,maxlen=max_length)
    
#     #print(X_text_train_padded)

#     # Tokenize and sequence validation data                   
#     X_text_test_sequences = tokenizer.texts_to_sequences(X_text_test)

#     # pad_sequences for validation set
#     X_text_test_padded = pad_sequences(X_text_test_sequences,maxlen=max_length)


#     f = open('glove.6B/glove.6B.100d.txt',encoding='utf-8')   
#     for line in f:       
#        values = line.split()       
#        word = values[0]       
#        coefs = np.asarray(values[1:], dtype='float32') 
#        embeddings_index[word] = coefs   
#     f.close()

#     # Creating embedding matrix for each word in Our corpus
#     embedding_matrix = np.zeros((len(word_index) + 1, 100))   
#     for word, i in word_index.items():   
#        embedding_vector = embeddings_index.get(word)   
#        if embedding_vector is not None:   
#            # words not found in the embedding index will be all-zeros.  
#            embedding_matrix[i] = embedding_vector
    
#     return X_text_train_padded,X_text_test_padded,embedding_matrix,word_index



# X_text_train_padded,X_text_test_padded,embedding_matrix,word_index = preProcessText(X_train_t, X_test_t)




# def make_model(max_length,embedding_matrix,word_index):

#     # Defining the embedding layer
#     max_length = 1000
#     embedding_dim = 64 
#     input1=Input(shape=(max_length,))
    
#     embedding_layer = Embedding(len(word_index) + 1,100,weights=[embedding_matrix],input_length=max_length,trainable=False)(input1)
                                
#     # Building LSTM for text features                          
#     bi_lstm_1 = Bidirectional(LSTM(embedding_dim,return_sequences=True))(embedding_layer)
    
#     bi_lstm_2 = Bidirectional(LSTM(embedding_dim))(bi_lstm_1)   
#     lstm_output =  Model(inputs = input1,outputs = bi_lstm_2)
    
#     #Inputting Number features
#     input2=Input(shape=(9,))  
    
#     # Merging inputs
#     merge = concatenate([lstm_output.output,input2])
    
#     # Building dense layers for regression with number features
#     reg_dense1 = Dense(64, activation='relu')(merge)
#     reg_dense2 = Dense(16, activation='relu')(reg_dense1) 
#     output1 = Dense(1, activation='sigmoid')(reg_dense2)
    
#     # Building dense layers for classification with number features
#     clf_dense1 = Dense(64, activation='relu')(merge)
#     clf_dense2 = Dense(16, activation='relu')(clf_dense1)
    
#     # 5 Categories in classification
#     output2 = Dense(5, activation='softmax')(clf_dense2)
 
#     model = Model(inputs=[lstm_output.input,input2], outputs=[output1,output2])
 
#     return model


# mlp = createMLP(X_train_n.shape[1])
# cnn = make_model(1000,embedding_matrix,word_index)

# combinedInput = concatenate([mlp.output, cnn.output[0]])
# # our final FC layer head will have two dense layers, the final one
# # being our regression head
# x = Dense(4, activation="relu")(combinedInput)
# x = Dense(1, activation="linear")(x)
# # our final model will accept categorical/numerical data on the MLP
# # input and images on the CNN input, outputting a single value (the
# # predicted price of the house)
# model = Model(inputs=[mlp.input, cnn.input], outputs=x)

# model.compile(loss="mean_absolute_percentage_error")

# model.fit(x=[X_train_n, X_text_train_padded], y=y_train_t,validation_data=([X_test_n, X_text_test_padded], y_test_t),epochs=200, batch_size=8)

# preds = model.predict([X_test_n, X_text_test_padded])


# diff = preds.flatten() - y_test_t
# percentDiff = (diff / y_test_t) * 100
# absPercentDiff = np.abs(percentDiff)

# mean = np.mean(absPercentDiff)
# std = np.std(absPercentDiff)

# print(mean, std)

# # Now we can train our model. Here we chose a Gradient Boosting Regressor and we set our loss function 
# reg = GradientBoostingRegressor()#reg = RandomForestRegressor() #
# #reg = LinearRegression()

# # We fit our model using the training data
# reg.fit(X_train, y_train)
# # And then we predict the values for our testing set
# y_pred = reg.predict(X_test)
# # We want to make sure that all predictions are non-negative integers
# y_pred = [int(value) if value >= 0 else 0 for value in y_pred]

# print("Prediction error:", mean_absolute_error(y_true=y_test, y_pred=y_pred))


# ###################################
# # Once we finalized our features and model we can train it using the whole training set and then produce prediction for the evaluating dataset
# ###################################
# # Load the evaluation data
# eval_data = pd.read_csv("evaluation.csv")
# # Transform our data into tfidf vectors
# vectorizer = TfidfVectorizer(max_features=100, stop_words=stopwords.words('french'))
# y_train = train_data['retweets_count']
# X_train = vectorizer.fit_transform(train_data['text'])
# # We fit our model using the training data
# reg = GradientBoostingRegressor()
# reg.fit(X_train, y_train)
# X_val = vectorizer.transform(eval_data['text'])
# # Predict the number of retweets for the evaluation dataset
# y_pred = reg.predict(X_val)
# y_pred = [int(value) if value >= 0 else 0 for value in y_pred]
# # Dump the results into a file that follows the required Kaggle template
# with open("gbr_predictions.txt", 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(["TweetID", "retweets_count"])
#     for index, prediction in enumerate(y_pred):
#         writer.writerow([str(eval_data['TweetID'].iloc[index]) , str(int(prediction))])

# dummy_regr = DummyRegressor(strategy="mean")
# dummy_regr.fit(X_train, y_train)
# dummy_pred = dummy_regr.predict(X_val)
# with open("mean_predictions.txt", 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(["TweetID", "retweets_count"])
#     for index, prediction in enumerate(dummy_pred):
#         writer.writerow([str(eval_data['TweetID'].iloc[index]) , str(int(prediction))])

# dummy_regr = DummyRegressor(strategy="constant", constant=0)
# dummy_regr.fit(X_train, y_train)
# dummy_pred = dummy_regr.predict(X_val)

# with open("zero_predictions.txt", 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(["TweetID", "retweets_count"])
#     for index, prediction in enumerate(dummy_pred):
#         writer.writerow([str(eval_data['TweetID'].iloc[index]) , str(int(prediction))])





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/30
461/461 [==============================] - 3s 5ms/step - loss: 10742401.0000 - val_loss: 737304.5000
Epoch 2/30
461/461 [==============================] - 2s 5ms/step - loss: 1871926.5000 - val_loss: 3028389.5000
Epoch 3/30
461/461 [==============================] - 2s 4ms/step - loss: 879511.5625 - val_loss: 1705988.8750
Epoch 4/30
461/461 [==============================] - 2s 4ms/step - loss: 985701.4375 - val_loss: 882865.6875
Epoch 5/30
461/461 [==============================] - 2s 4ms/step - loss: 458050.8125 - val_loss: 226068.7344
Epoch 6/30
461/461 [==============================] - 2s 4ms/step - loss: 326895.4062 - val_loss: 64150.4297
Epoch 7/30
461/461 [==============================] - 2s 4ms/step - loss: 275336.9688 - val_loss: 382559.3438
Epoch 8/30
461/461 [==============================] - 2s 4ms/step - loss: 255266.8125 - val_loss: